In [1]:
def readin(filename):
    with open(filename) as f:
        lines = f.readlines()

    #something to remove \n 
    str_lines = []
    for l in lines:
        if l[-1:]=="\n":
            str_lines.append(l[0:-1])
        else: 
            str_lines.append(l)

    #check all was fine
    #print(lines[-2:])
    #print(str_lines[-2:])
    return str_lines

elf_data = readin("Day13input.txt")

# --- Day 13: Distress Signal ---

You climb the hill and again try contacting the Elves. However, you instead receive a signal you weren't expecting: a distress signal.

Your handheld device must still not be working properly; the packets from the distress signal got decoded out of order. You'll need to re-order the list of received packets (your puzzle input) to decode the message.

Your list consists of pairs of packets; pairs are separated by a blank line. You need to identify how many pairs of packets are in the right order.



In [2]:
def parse_input(data):
    received_packets = []
    for l in range(0, len(data), 3):
        received_packets.append((eval(data[l]), eval(data[l+1])))
    return received_packets

parsed_elf_data = parse_input(elf_data)
#print(len(parsed_elf_data))
#print(len(elf_data))
#print(parsed_elf_data[-1:])




For example:

    [1,1,3,1,1]
    [1,1,5,1,1]

    [[1],[2,3,4]]
    [[1],4]

    [9]
    [[8,7,6]]

    [[4,4],4,4]
    [[4,4],4,4,4]

    [7,7,7,7]
    [7,7,7]

    []
    [3]

    [[[]]]
    [[]]

    [1,[2,[3,[4,[5,6,7]]]],8,9]
    [1,[2,[3,[4,[5,6,0]]]],8,9]
    


In [3]:
test_data = ["[1,1,3,1,1]",
"[1,1,5,1,1]",
"",
"[[1],[2,3,4]]",
"[[1],4]",
"",
"[9]",
"[[8,7,6]]",
"",
"[[4,4],4,4]",
"[[4,4],4,4,4]",
"",
"[7,7,7,7]",
"[7,7,7]",
"",
"[]",
"[3]",
"",
"[[[]]]",
"[[]]",
"",
"[1,[2,[3,[4,[5,6,7]]]],8,9]",
"[1,[2,[3,[4,[5,6,0]]]],8,9]"]

parsed_test_data = parse_input(test_data)


Packet data consists of lists and integers. Each list starts with [, ends with ], and contains zero or more comma-separated values (either integers or other lists). Each packet is always a list and appears on its own line.

When comparing two values, the first value is called *left* and the second value is called *right*. Then:

- If *both values are integers*, the *lower integer* should come first. If the left integer is lower than the right integer, the inputs are in the right order. If the left integer is higher than the right integer, the inputs are not in the right order. Otherwise, the inputs are the same integer; continue checking the next part of the input.
- If *both values are lists*, compare the first value of each list, then the second value, and so on. If the left list runs out of items first, the inputs are in the right order. If the right list runs out of items first, the inputs are not in the right order. If the lists are the same length and no comparison makes a decision about the order, continue checking the next part of the input.
- If *exactly one value is an integer*, convert the integer to a list which contains that integer as its only value, then retry the comparison. For example, if comparing [0,0,0] and 2, convert the right value to [2] (a list containing 2); the result is then found by instead comparing [0,0,0] and [2].


In [4]:
def check_packets(parsed_data):
    index = 0
    sum = 0
    for (list_l, list_r) in parsed_data:
        index = index + 1
        x = check_l_and_r(list_l, list_r) 
        print("Pair ", index, list_l, list_r, "is ", x)
        if x is True:
            sum = sum+index
    return sum

def check_l_and_r(list_l, list_r):
    #print("Check", list_l, list_r)
    lt = type(list_l)
    rt = type(list_r)
    if lt != rt:
        if isinstance(list_l, int):
            #print("1")
            return check_l_and_r([list_l], list_r)
        else: # assume rt is also  int:
            #print("2")
            return check_l_and_r(list_l, [list_r])
    elif isinstance(list_l, int): # so by default, so is rt
        if int(list_l) < int(list_r):
            #print("3")
            return True
        elif int(list_l) > int (list_r):
            #print("3a")
            return False
        else:  #they are equal
            #print("4") 
            return None 
    else: #assume both are lists
        correct = None
        #list_l = make_list_from_list(list_l)
        #list_r = make_list_from_list(list_r)
        i=0
        while len(list_l)>i and len(list_r)>i and correct is None:
            correct = check_l_and_r(list_l[i], list_r[i])
            if not correct:
                i = i+1
        if correct is not None:    # HERE
            #print("5")
            return correct
        elif len(list_l) == i and len(list_r) == i:
            #print("6")
            return None
        else: 
            #print("7")
            return len(list_l) == i or len(list_r) != i
        

In [5]:
check_packets(parsed_test_data)

Pair  1 [1, 1, 3, 1, 1] [1, 1, 5, 1, 1] is  True
Pair  2 [[1], [2, 3, 4]] [[1], 4] is  True
Pair  3 [9] [[8, 7, 6]] is  False
Pair  4 [[4, 4], 4, 4] [[4, 4], 4, 4, 4] is  True
Pair  5 [7, 7, 7, 7] [7, 7, 7] is  False
Pair  6 [] [3] is  True
Pair  7 [[[]]] [[]] is  False
Pair  8 [1, [2, [3, [4, [5, 6, 7]]]], 8, 9] [1, [2, [3, [4, [5, 6, 0]]]], 8, 9] is  False


13


Using these rules, you can determine which of the pairs in the example are in the right order:

    == Pair 1 ==
    - Compare [1,1,3,1,1] vs [1,1,5,1,1]
    - Compare 1 vs 1
    - Compare 1 vs 1
    - Compare 3 vs 5
        - Left side is smaller, so inputs are in the right order

    == Pair 2 ==
    - Compare [[1],[2,3,4]] vs [[1],4]
    - Compare [1] vs [1]
        - Compare 1 vs 1
    - Compare [2,3,4] vs 4
        - Mixed types; convert right to [4] and retry comparison
        - Compare [2,3,4] vs [4]
        - Compare 2 vs 4
            - Left side is smaller, so inputs are in the right order

    == Pair 3 ==
    - Compare [9] vs [[8,7,6]]
    - Compare 9 vs [8,7,6]
        - Mixed types; convert left to [9] and retry comparison
        - Compare [9] vs [8,7,6]
        - Compare 9 vs 8
            - Right side is smaller, so inputs are not in the right order

    == Pair 4 ==
    - Compare [[4,4],4,4] vs [[4,4],4,4,4]
    - Compare [4,4] vs [4,4]
        - Compare 4 vs 4
        - Compare 4 vs 4
    - Compare 4 vs 4
    - Compare 4 vs 4
    - Left side ran out of items, so inputs are in the right order

    == Pair 5 ==
    - Compare [7,7,7,7] vs [7,7,7]
    - Compare 7 vs 7
    - Compare 7 vs 7
    - Compare 7 vs 7
    - Right side ran out of items, so inputs are not in the right order

    == Pair 6 ==
    - Compare [] vs [3]
    - Left side ran out of items, so inputs are in the right order

    == Pair 7 ==
    - Compare [[[]]] vs [[]]
    - Compare [[]] vs []
        - Right side ran out of items, so inputs are not in the right order

    == Pair 8 ==
    - Compare [1,[2,[3,[4,[5,6,7]]]],8,9] vs [1,[2,[3,[4,[5,6,0]]]],8,9]
    - Compare 1 vs 1
    - Compare [2,[3,[4,[5,6,7]]]] vs [2,[3,[4,[5,6,0]]]]
        - Compare 2 vs 2
        - Compare [3,[4,[5,6,7]]] vs [3,[4,[5,6,0]]]
        - Compare 3 vs 3
        - Compare [4,[5,6,7]] vs [4,[5,6,0]]
            - Compare 4 vs 4
            - Compare [5,6,7] vs [5,6,0]
            - Compare 5 vs 5
            - Compare 6 vs 6
            - Compare 7 vs 0
                - Right side is smaller, so inputs are not in the right order

What are the indices of the pairs that are already in the right order? (The first pair has index 1, the second pair has index 2, and so on.) In the above example, the pairs in the right order are 1, 2, 4, and 6; the sum of these indices is 13.

Determine which pairs of packets are already in the right order. What is the sum of the indices of those pairs?

In [6]:
check_packets(parsed_elf_data)

Pair  1 [[[[4, 7, 7], 0, 4, [6, 3, 3, 7, 10], 2], 2, [[3]]], []] [[[[1, 4, 8], [3, 3, 1, 4]]]] is  False
Pair  2 [[7, 6], [], [[5], 0, 10, [7, 9, [7], 0]]] [[[[6, 9, 0]], 0]] is  False
Pair  3 [[7, 7, [[5, 1, 4, 9, 4], 8, [8], [1, 1, 6, 2]], [[], 3, 6, [], []]], [], [10, 2, 7], [0, [1, 2, 4, 1, 0], [1, [], 0], 5, []], [4, [[], [], []], []]] [[3, 6], [[9, 6, []], 8, [8, 8, 3, [8, 2, 3, 10], [6, 5]], [6, 4, [8, 5], 8]]] is  False
Pair  4 [[[[4, 8, 1, 3, 4], [2, 4, 10, 5, 4], [7, 7], 7], [[9, 8, 4, 7, 0]], 8, [3, 1]], [[], [[9, 3], 5], [], 7, [[], 8, 8, [6, 0, 3, 10, 2], [0, 8, 7, 0]]], [9, [4], 7, 3]] [[5], [[[], [], [7, 10]], 3, 6], [[[], 4, [8, 6, 3, 1], 0, [5, 1, 0, 4]], [], [[9, 5]], 4]] is  True
Pair  5 [[6, 10, 9, [], [[2]]], [9, [10, [3, 5, 0], [6, 5]]], [], [[8, [2, 2, 3]], 6]] [[], [[], 9, [9, 2, [6, 8, 10, 3], 2], 9, 3]] is  False
Pair  6 [[[[3], []], 5], [[], [7, [3, 3, 3], 2, [1], [6, 7, 9]], [], 8, 1], [9, [0, 0, [5, 3, 5, 1], [2], 2], 3], [2, [0, 4]]] [[[]], [[[], 10, [8, 0

6420